In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from analyze import get_events_by_time
from datetime import datetime, timedelta
from parse import parse_follows
import pandas as pd
import json
from ingest import parse_mention_tags, parse_reply_tags, parse_user_metadata
from nostr_sdk import Keys, Client, EventBuilder, Filter, PublicKey, EventId, Options, Timestamp
from datetime import timedelta
import numpy as np
import time
from query import query_events, query_db
from utils import postprocess

from sqlalchemy import create_engine, Column, BigInteger, String
from sqlalchemy.dialects.postgresql import JSONB
from sqlalchemy.orm import declarative_base, sessionmaker
from sqla_models import User, Follow
import plotly.express as px

pablo_npub = "npub1l2vyh47mk2p0qlsku7hg0vn29faehy9hy34ygaclpn66ukqp3afqutajft"
pablo_hex = PublicKey.from_bech32(pablo_npub).to_hex()
local_npub1 = "npub1ujm8l8mupgwvu8pye2ged78pg3huectlmm6at66x5wffgvl2fkgsygt8l4"
# local_npub2 = "npub18cd2agsh7qgju43zr9qwduggd4vnqtpyhzremtr6lwgpm3ue3t3qlx2snh"



In [ ]:
engine = create_engine(
    "postgresql://postgres@localhost:5432/postgres"
)
Session = sessionmaker(bind=engine)

In [ ]:
keys = Keys.generate()
# print(keys.public_key().to_bech32())
opts = Options().timeout(timedelta(seconds=30))
client = Client.with_opts(keys, opts)
# client = Client(keys)
client.add_relay("wss://relay.damus.io")
client.add_relay("wss://nostr.oxtr.dev")
client.add_relay('wss://relay.nostr.band')
client.add_relay('wss://relay.primal.net')
client.add_relay('wss://relay.mostr.pub')
client.connect()
# PublicKey.from_hex('031ea306188fee631a86f1a0e49cee8f3d62592b2e7be5f8eb1788e327db340a').to_bech32()

In [ ]:
PublicKey.from_hex("e4b67f9f7c0a1cce1c24ca9196f8e1446fcce17fdef5d5eb46a3929433ea4d91").to_bech32()
npub18cd2agsh7qgju43zr9qwduggd4vnqtpyhzremtr6lwgpm3ue3t3qlx2snh

In [ ]:
df = query_events(
    client, kind=1, npub=pablo_npub, num_days=5, num_limit=None, timeout_secs=30
)

In [ ]:
filter = Filter().limit(100000)
all_events = client.get_events_of([filter], None)
df_all = pd.DataFrame([json.loads(event.as_json()) for event in all_events]).set_index("id")

In [ ]:
df_all = pd.read_pickle('df_all.pkl')

In [ ]:
PublicKey.from_hex("ee6ea13ab9fe5c4a68eaf9b1a34fe014a66b40117c50ee2a614f4cda959b6e74").to_bech32()

In [ ]:
# df_all = pd.read_pickle('df_all.pkl')
# df_reply = parse_reply_tags(df_all)
# df_mention = parse_mention_tags(df_all)
# df_reply.to_sql('reply', engine, if_exists="replace")
# df_mention.to_sql('mention', engine, if_exists="replace")
df_user = parse_user_metadata(df_all)
user_cols = [col.name for col in inspect(User).columns]
user_cols = [col for col in user_cols if col != "id"]
df_user[user_cols].to_sql('user', engine, if_exists="replace")

In [ ]:
row = df_user.loc['91fd1de594297ff4cdeae5a58aa7be91dd281bd0df56ae9af191f5add3372e30']

In [ ]:
# # Filter().event(event_id)
# from datetime import datetime, timedelta

# # Calculate the timestamp for 1 day ago
# one_day_ago = datetime.now() - timedelta(days=1)
# filter = Filter().author(pablo_hex).since(Timestamp.from_secs(int(one_day_ago.timestamp())))

filter = Filter().author(pablo_hex).since(get_filter_timestamp(num_days=1))
pablo_events = client.get_events_of([filter], None)
df_pablo = pd.DataFrame([json.loads(event.as_json()) for event in pablo_events]).set_index("id")

In [ ]:
df_kind_3 = query_db(
    Session=Session,
    kind=3
)
df_follows = parse_follows(df_kind_3)
df_follows.to_sql('follow', engine, if_exists="replace")

In [ ]:
df_local = query_db(
    Session=Session,
    npub=local_npub1,
)

In [ ]:
PublicKey.from_hex("5cad82c898ee66013711945d687f7d9549f645a0118467dae2f5e274c598d6ff").to_bech32()
# PublicKey.from_hex("25e5c82273a271cb1a840d0060391a0bf4965cafeb029d5ab55350b418953fbb").to_bech32()

In [ ]:
# read sql table with sqla
with Session() as session:
    results = session.query(Follow).all()
df = pd.DataFrame([r.__dict__ for r in results])
df = df.drop('_sa_instance_state', axis=1)


# read sql table with raw sql query
# sql_query = "SELECT * FROM events"
# sql_query = "SELECT * FROM events WHERE tags ? 't'"
sql_query = "SELECT * FROM events WHERE tags->>'t' = 'sm41623576'";

df_results = pd.read_sql(sql_query, engine)

In [ ]:
df_local.columns

In [ ]:
df_local = get_events_by_time(df_local)
df_local[["created_at", "day_of_week", "hour_of_day"]]

In [ ]:
PublicKey.from_hex('22dd8df1fed1da2574c4917146d93dcb679549aeead8f98cbbaf166d183662ad').to_bech32()

In [ ]:
PublicKey.from_hex('3e1aaea217f0112e56221940e6f1086d59302c24b8879dac7afb901dc7998ae2').to_bech32()


In [ ]:
len(all_events)

In [ ]:
pd.to_pickle(df_all, 'df_all.pkl')

In [ ]:
# df = pd.read_pickle('df_all.pkl')
df_all.to_sql('event', engine, if_exists="replace", dtype={'tags': JSONB,})

In [ ]:
users = df.pubkey.unique()[:10]
df_temp = df[df.pubkey.isin(users)]

In [ ]:
EventId.from_hex("be169dcde5d3a423916827449f37bd09be0e2cbb34ddf62204d6c0ba97d03a0c").to_bech32()

In [ ]:
df_reply = parse_reply_tags(df)
df_mention = parse_mention_tags(df)

In [ ]:
df_reply.columns, df_mention.columns

In [ ]:
df_mention

In [ ]:
dft = df[df.pubkey.isin(top_pubkeys)]

In [ ]:
top_pubkeys = df.groupby('pubkey').size().sort_values(ascending=False).index[5:10]

In [ ]:
dft.groupby(['pubkey', 'kind']).size().reset_index()

In [ ]:
df_mention.dtypes

In [ ]:
dft.kind = dft.kind.astype(str)
px.histogram(dft, x="pubkey", color="kind")

# fig = px.histogram(
#     x=df.kind.astype(str), title=f"Histogram of events by kind for {npub_value}"
# )

# px.histogram(dfx, x="kind", y="count", color="pubkey", pattern_shape="nation", pattern_shape_sequence=[".", "x", "+"])
# px.bar(
#     dft.groupby(['pubkey', 'kind']).size().reset_index(),
#     )

In [ ]:
df.groupby('pubkey').size().sort_values(ascending=False)[:10]

In [ ]:
df.columns

In [ ]:
df_mention.columns

In [ ]:
df_temp

In [ ]:
# df = px.data.tips()

# fig = px.histogram(df, x="total_bill", y="tip", color="sex", marginal="rug", hover_data=df.columns)
# fig.show()

fig = px.histogram(df_temp, x="pubkey", y="count", color="kind", pattern_shape="nation", pattern_shape_sequence=[".", "x", "+"])
# fig = px.histogram(df, x="total_bill", y="tip", color="sex", marginal="rug", hover_data=df.co lumns)
fig.show()

In [ ]:
# EventId.from_hex('3f770d65d3a764a9c5cb503ae123e62ec7598ad035d836e2a810f3877a745b24').to_bech32()
# PublicKey.from_hex('3f770d65d3a764a9c5cb503ae123e62ec7598ad035d836e2a810f3877a745b24').to_bech32()

In [ ]:
grp = df_reply.groupby('event_id').size().reset_index(name='count')
grp.sort_values('count', ascending=False)['event_id'].iloc[0]

In [ ]:
# thread
df_reply[df_reply.event_id == orig_id].sort_values('created_at', ascending=True)['source_id'].iloc[0]

In [ ]:
pablo = "npub1l2vyh47mk2p0qlsku7hg0vn29faehy9hy34ygaclpn66ukqp3afqutajft"
PublicKey.from_bech32(pablo).to_hex()

In [ ]:
pablo = "npub1l2vyh47mk2p0qlsku7hg0vn29faehy9hy34ygaclpn66ukqp3afqutajft"
pk = PublicKey.from_bech32(pablo)
filter = Filter().author(pk.to_hex()).limit(50000)

events_pablo = client.get_events_of([filter], timedelta(seconds=1000))
df_pablo = pd.DataFrame([json.loads(event.as_json()) for event in events_pablo]).set_index("id")

In [ ]:
def get_events(npub):
    # Subtract 7 days from the current date and time
    start_ts = (datetime.now() - timedelta(days=7)).timestamp()
    # timestamp = int(seven_days_ago.timestamp())
    pk = PublicKey.from_bech32(npub)
    filter = Filter().author(pk.to_hex()).since(start_ts)
    event = client.get_events_of([filter], timedelta(seconds=1000))

In [ ]:
df_pablo = pd.read_pickle('pablo.pkl')

In [ ]:
# pd.to_pickle(df_pablo, 'pablo.pkl')
len(df_pablo)

In [ ]:
temp = df_pablo.loc['5ff739f0d1998ed71a52e45d043a1a98d95aa4cac4072d7046cc01df448b5227'].iloc[0]['tags'][0]

In [ ]:
# df.to_sql('events', engine, if_exists="replace", dtype={
#     'tags': JSONB,
#     'tags_relay_url': JSONB,
# })

# df_e.to_sql('replys', engine, if_exists="replace")
# df_p.to_sql('mentions', engine, if_exists="replace")

In [ ]:
df_pablo.columns

In [ ]:
df_reply.columns

In [ ]:
df_mention.columns

In [ ]:
grp = df_e.groupby('event_id').size().reset_index(name='count')

In [ ]:
grp.sort_values('count', ascending=False)['event_id'].iloc[2]

In [ ]:
# EventId.from_hex('ef53730b18bc0a77db1078f66bebe516a13e10e036357e2750e7890da1ae047c').to_bech32()
EventId.from_hex('b6b186d3a97cb621ada7ffea9f26589c32643802b012024f4a1f2bc850faef5b') # should be nevent1 not note1


In [ ]:
EventId.from_hex('b6b186d3a97cb621ada7ffea9f26589c32643802b012024f4a1f2bc850faef5b').to_bech32()

In [ ]:
df_e[df_e.event_id == 'a6f436a59fdb5e23c757b1e30478742996c54413df777843e0a731af56a96eea']['source_id'].iloc[0]

In [ ]:
results = pd.read_sql("select * from replys", engine)

In [ ]:
df_e.head()

In [ ]:
results

In [ ]:
df_e['e_col4'].unique()

In [ ]:
df_p['p_col3'].unique()

In [ ]:
df_pablo.head()

In [ ]:
tags_count = []
for item in df_pablo['tags']:
    print(item)
    break
# result = [(lst[0], len(lst)) if lst else (None, 0) for lst in df_pablo['tags']]

# print(result)

In [ ]:
result[0]

In [ ]:
df_pablo.iloc[1]['tags']

In [ ]:
# def query_events(client, kind=None, num_limit=1000):
filter = Filter().limit(50000)
events = client.get_events_of([filter], timedelta(seconds=1000))
df = pd.DataFrame([json.loads(event.as_json()) for event in events]).set_index("id")

In [ ]:
out = df['tags'].apply(lambda x: x.keys() if x else None)

In [ ]:
df.tags.iloc[0]

In [ ]:
from collections import defaultdict
all_tags = defaultdict(int)
for row in out.values:
    if row is None:
        continue
    for item in list(row):
        all_tags[item] += 1

In [ ]:
all_tags

In [ ]:
len(all_tags)

In [ ]:
df.iloc[0]['pubkey']

In [ ]:
# Using the `size` method which returns the count of each group
grouped = df.groupby('pubkey').size().reset_index(name='count')

# OR using the `count` method
# grouped = df.groupby('kind')['kind'].count().reset_index(name='count')


In [ ]:
grouped.sort_values('count', ascending=False).head(10)

In [ ]:
# pd.to_pickle(df, 'events_50000.pkl')
# df = pd.read_pickle('events_100.pkl')

In [ ]:
PublicKey.from_hex(df_pablo.pubkey.unique()[0]).to_bech32()

In [ ]:
for i in range(20):
    print(df_pablo.iloc[i]['tags'])

In [ ]:
df_pablo.to_sql('pablo', engine, if_exists="replace", dtype={'tags': JSONB})

In [ ]:
# df = query_events(client)
# loses association with key (e or p may have different relay url)
df["tags_relay_url"] = df["tags"].apply(
    lambda x: {item[0]: item[2] for item in x if len(item) > 2} or None
)
df['tags'] = df['tags'].apply(lambda x: {item[0]: item[1] if len(item) > 1 else None for item in x} or None)

df.to_sql('events', engine, if_exists="replace", dtype={
    'tags': JSONB,
    'tags_relay_url': JSONB,
})

# sql_query = "SELECT * FROM events"
# sql_query = "SELECT * FROM events WHERE tags ? 't'"
# sql_query = "SELECT * FROM events;"

# df_results = pd.read_sql(sql_query, engine)


In [ ]:
# Create a SQLAlchemy engine and session factory
Session = sessionmaker(bind=engine)

# Define the ORM class for the 'events' table
Base = declarative_base()

class Events(Base):
    __tablename__ = "events"
    id = Column(String, primary_key=True)
    created_at = Column(BigInteger)
    kind = Column(BigInteger)
    tags = Column(JSONB)
    tags_relay_url = Column(JSONB)
    pubkey = Column(String)
    content = Column(String)
    sig = Column(String)


In [ ]:
df.groupby('pubkey').size()

In [ ]:
# Create a session and query the table using the ORM
with Session() as session:
    results = session.query(Events).all()

# Convert the results into a pandas DataFrame
df = pd.DataFrame([r.__dict__ for r in results])

# Removing unnecessary meta-data related columns
# df = df.drop('_sa_instance_state', axis=1)

print(df)

In [ ]:
npub = "npub1dergggklka99wwrs92yz8wdjs952h2ux2ha2ed598ngwu9w7a6fsh9xzpc"
pk = PublicKey.from_bech32(npub)
pk.to_hex()

In [ ]:
event_id = EventId.from_hex(df.loc[0, 'id'])
event_id.to_bech32()

In [ ]:
event_id.to_bech32()

In [ ]:
type(keys), type(keys.public_key()), type(keys.public_key().to_bech32())

In [ ]:
# create kind name -> kind val dict
# handle 3rd field (recommended relay url optional) in tag list (see nips1)
# get events for all kinds
# get zaps
# large query

In [ ]:
def query_zaps():
    filter = Filter().kind(9735).limit(10)
    events = client.get_events_of([filter], timedelta(seconds=10))
    return pd.DataFrame([json.loads(event.as_json()) for event in events])